# Case Study AirBnB Forecasting

In [1]:
# Data Representation
import pandas as pd

# Visualization
import plotly.express as px
import plotly
import plotly.graph_objects as go

random_state = 42
pd.set_option('display.max_rows', 100)

Inspiration --> https://www.kaggle.com/nikomata/time-series-forecasting-airbnb-data-fbprophet

In [2]:
link = 'https://drive.google.com/file/d/1OxHShzKji6vodNnTkw1bNZyTaM5hv7mC/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+link.split('/')[-2]

In [3]:
calendar_df = pd.read_csv(path)
calendar_df.head(1)

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00


In [4]:
calendar_df.head(1).T

,0
listing_id,241032
date,2016-01-04
available,t
price,$85.00


In [5]:
calendar_df.shape

(1393570, 4)

In [6]:
# % of NaN values
missing_series = pd.Series(1 - calendar_df.count() / len(calendar_df))
missing_series

listing_id    0.00000
date          0.00000
available     0.00000
price         0.32939
dtype: float64

> Since there's just a few prices missing and there's a lot of observations (1,393,570), we will just drop those missing observations

In [7]:
calendar_df.dropna(subset=['price'], inplace=True)

In [8]:
calendar_df.shape

(934542, 4)

In [9]:
calendar_df['price_float'] = calendar_df['price'].replace('[\$\,]',"",regex=True).astype(float)

In [10]:
calendar_df.dtypes

listing_id       int64
date            object
available       object
price           object
price_float    float64
dtype: object

## Segmenting Time Series Data into Temporal Periodicity
- Daily
- Monthly
- Quarterly

In [12]:
calendar_df.shape

(934542, 5)

In [13]:
calendar_df['date'].value_counts().head(10)

2017-01-01    2922
2016-12-31    2859
2016-12-30    2840
2016-12-29    2835
2016-12-28    2833
2016-12-27    2831
2016-12-26    2831
2016-12-25    2829
2016-12-24    2822
2016-12-23    2822
Name: date, dtype: int64

In [14]:
calendar_df['date'].value_counts().mean()

2560.38904109589

> This tells us that across this dataset, there are on average 2,560 listing prices per a given day.

In [15]:
daily_average_prices_df = calendar_df.groupby('date')[["price_float"]].mean()
daily_average_prices_df.columns = ['Average']
daily_average_prices_df.index = pd.to_datetime(daily_average_prices_df.index)
daily_average_prices_df.head()

,Average
date,
2016-01-04,122.085879
2016-01-05,120.681307
2016-01-06,120.928258
2016-01-07,120.958896
2016-01-08,127.640853


In [16]:
daily_average_prices_df.shape

(365, 1)

In [17]:
daily_average_prices_df.index.dtype

dtype('<M8[ns]')

In [18]:
monthly_average_prices_df = daily_average_prices_df[['Average']].resample('M').mean()
monthly_average_prices_df.head()

,Average
date,
2016-01-31,121.692505
2016-02-29,124.315614
2016-03-31,128.640797
2016-04-30,135.109460
2016-05-31,139.539566


In [19]:
monthly_average_prices_df.shape

(13, 1)

In [20]:
quarterly_average_prices_df = daily_average_prices_df[['Average']].resample('Q').mean()
quarterly_average_prices_df.head()

,Average
date,
2016-03-31,125.004632
2016-06-30,140.693254
2016-09-30,148.737780
2016-12-31,136.663931
2017-03-31,136.849867


In [21]:
quarterly_average_prices_df.shape # We see that the data is spread across 5 quarters

(5, 1)

In [57]:
px.line(daily_average_prices_df,
        labels={'value':'Average Price Per Night ($)', 'date':'Date'},
        title="Daily Averages Across Time")

In [56]:
#help(daily_average_prices_df.index)

In [51]:
daily_average_prices_df.index.day_name()

Index(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday', 'Monday', 'Tuesday', 'Wednesday',
       ...
       'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday', 'Monday'],
      dtype='object', name='date', length=365)

In [ ]:
px.line(daily_average_prices_df,
        labels={'value':'Average Price Per Night ($)', 'date':'Date'},
        color=daily_average_prices_df.index.day_name(),
        title="Monthly Averages Across Time")

In [23]:
px.line(monthly_average_prices_df,
        labels={'value':'Average Price Per Night ($)', 'date':'Date'},
        title="Monthly Averages Across Time")

In [24]:
px.line(quarterly_average_prices_df,
        labels={'value':'Average Price Per Night ($)', 'date':'Date'},
        title="Quarterly Averages Across Time")

## Let's Forecast
We will be using Facebook's Prophet time series modeling library.

### Forecast Daily Averages

In [25]:
from fbprophet import Prophet

In [26]:
prophet = Prophet(interval_width = 0.95,
                  daily_seasonality = True,
                #   weekly_seasonality = True,
                  yearly_seasonality = True,
                  changepoint_prior_scale = 0.095
                  )
prophet

`prophet` expects to fit a dataframe with columns: df[['ds', 'y']]

In [27]:
# we are going to use the df w/daily averages
calendar_prophet_df = daily_average_prices_df.copy()
calendar_prophet_df.reset_index(inplace=True)
calendar_prophet_df.columns = ['ds', 'y']
calendar_prophet_df.head()

,ds,y
0,2016-01-04,122.085879
1,2016-01-05,120.681307
2,2016-01-06,120.928258
3,2016-01-07,120.958896
4,2016-01-08,127.640853


In [28]:
prophet.fit(calendar_prophet_df)

In [29]:
y_pred_future = prophet.make_future_dataframe(periods = 60, freq = 'd')
y_pred_future['cap'] = 5.05
forecast = prophet.predict(y_pred_future)

In [30]:
forecast.head()

,ds,trend,cap,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-01-04,123.495795,5.05,118.520191,123.241553,123.495795,123.495795,-2.672515,-2.672515,-2.672515,...,-2.164526,-2.164526,-2.164526,-6.438030,-6.438030,-6.438030,0.0,0.0,0.0,120.823281
1,2016-01-05,123.543654,5.05,118.108256,123.011871,123.543654,123.543654,-3.003575,-3.003575,-3.003575,...,-2.431518,-2.431518,-2.431518,-6.502100,-6.502100,-6.502100,0.0,0.0,0.0,120.540079
2,2016-01-06,123.591512,5.05,118.029149,122.814463,123.591512,123.591512,-3.112452,-3.112452,-3.112452,...,-2.449518,-2.449518,-2.449518,-6.592975,-6.592975,-6.592975,0.0,0.0,0.0,120.479061
3,2016-01-07,123.639371,5.05,118.905818,123.826789,123.639371,123.639371,-2.298657,-2.298657,-2.298657,...,-1.519862,-1.519862,-1.519862,-6.708837,-6.708837,-6.708837,0.0,0.0,0.0,121.340714
4,2016-01-08,123.687229,5.05,125.271740,130.524418,123.687229,123.687229,4.098942,4.098942,4.098942,...,5.016244,5.016244,5.016244,-6.847345,-6.847345,-6.847345,0.0,0.0,0.0,127.786171


In [31]:
previous_future_prices = forecast.set_index('ds').join(calendar_prophet_df.set_index('ds'))
previous_future_prices = previous_future_prices[['y', 'yhat', 'yhat_upper', 'yhat_lower']]
previous_future_prices.head()

,y,yhat,yhat_upper,yhat_lower
ds,,,,
2016-01-04,122.085879,120.823281,123.241553,118.520191
2016-01-05,120.681307,120.540079,123.011871,118.108256
2016-01-06,120.928258,120.479061,122.814463,118.029149
2016-01-07,120.958896,121.340714,123.826789,118.905818
2016-01-08,127.640853,127.786171,130.524418,125.271740


In [32]:
# help(go.Scatter)

In [33]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['y'], 
                mode='lines+markers',
                name='y',
                line_color='blue')
)

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['yhat'], 
                mode='lines+markers',
                line_color='red',
                name='yhat'),
)

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['yhat_upper'],
                fillcolor='rgba(0,0,0,.2)', 
                #opacity=0.1,
                fill='tonexty', 
                mode='none', 
                name='upper')
)

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['yhat_lower'],
                fillcolor='rgba(0,0,0,.2)', 
                #opacity=0.01,
                fill='tonexty', 
                mode='none', 
                name='lower')
)

fig.update_layout(
    title="Past, Present, and Predicted Future Daily Average Prices",
    xaxis_title="Date",
    yaxis_title="Average Price Per Night ($)",
    legend_title="Legend",
)


fig.show()

In [34]:
px.line(previous_future_prices,
        labels={'value':'Average Price Per Night ($)', 'ds':'Date'},
        title="Past, Present, and Predicted Future Daily Average Prices")

### Forecast Monthly Averages

In [35]:
prophet = Prophet(interval_width = 0.95,
                  daily_seasonality = True,
                  weekly_seasonality = True,
                  changepoint_prior_scale = 0.095
                  )
prophet

In [36]:
# we are going to use the df w/daily averages
calendar_prophet_df = monthly_average_prices_df.copy()
calendar_prophet_df.reset_index(inplace=True)
calendar_prophet_df.columns = ['ds', 'y']
calendar_prophet_df.head()

,ds,y
0,2016-01-31,121.692505
1,2016-02-29,124.315614
2,2016-03-31,128.640797
3,2016-04-30,135.109460
4,2016-05-31,139.539566


In [37]:
calendar_prophet_df.shape

(13, 2)

In [38]:
prophet.fit(calendar_prophet_df)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 9.


In [39]:
y_pred_future = prophet.make_future_dataframe(periods = 10, freq = 'm')
y_pred_future['cap'] = 5.05
forecast = prophet.predict(y_pred_future)

In [40]:
forecast.head()

,ds,trend,cap,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-01-31,7.078695,5.05,118.286735,125.858550,7.078695,7.078695,114.986149,114.986149,114.986149,113.261867,113.261867,113.261867,1.724281,1.724281,1.724281,0.0,0.0,0.0,122.064843
1,2016-02-29,11.805638,5.05,118.290891,125.855676,11.805638,11.805638,110.501116,110.501116,110.501116,113.261867,113.261867,113.261867,-2.760751,-2.760751,-2.760751,0.0,0.0,0.0,122.306754
2,2016-03-31,16.858577,5.05,126.833345,133.871575,16.858577,16.858577,113.777402,113.777402,113.777402,113.261867,113.261867,113.261867,0.515535,0.515535,0.515535,0.0,0.0,0.0,130.635980
3,2016-04-30,21.750014,5.05,131.899670,139.821022,21.750014,21.750014,113.742929,113.742929,113.742929,113.261867,113.261867,113.261867,0.481062,0.481062,0.481062,0.0,0.0,0.0,135.492944
4,2016-05-31,26.804499,5.05,137.754815,145.143388,26.804499,26.804499,114.616051,114.616051,114.616051,113.261867,113.261867,113.261867,1.354183,1.354183,1.354183,0.0,0.0,0.0,141.420550


In [41]:
previous_future_prices = forecast.set_index('ds').join(calendar_prophet_df.set_index('ds'))
previous_future_prices = previous_future_prices[['y', 'yhat', 'yhat_upper', 'yhat_lower']]
previous_future_prices.head()

,y,yhat,yhat_upper,yhat_lower
ds,,,,
2016-01-31,121.692505,122.064843,125.858550,118.286735
2016-02-29,124.315614,122.306754,125.855676,118.290891
2016-03-31,128.640797,130.635980,133.871575,126.833345
2016-04-30,135.109460,135.492944,139.821022,131.899670
2016-05-31,139.539566,141.420550,145.143388,137.754815


In [42]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['y'], 
                mode='lines+markers',
                name='y',
                line_color='blue')
)

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['yhat'], 
                mode='lines+markers',
                line_color='red',
                name='yhat'),
)

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['yhat_lower'],
                fillcolor='rgba(0,0,0,.2)', 
                fill='tonexty', 
                mode='none', 
                name='lower')
)

fig.add_trace(
    go.Scatter(x=previous_future_prices.index, 
                y=previous_future_prices['yhat_upper'],
                fillcolor='rgba(0,0,0,.2)', 
                fill='tonexty', 
                mode='none', 
                name='upper')
)



fig.update_layout(
    title="Past, Present, and Predicted Future Monthly Average Prices",
    xaxis_title="Date",
    yaxis_title="Average Price Per Night ($)",
    legend_title="Legend",
)


fig.show()

In [43]:
px.line(previous_future_prices,
        title="Past, Present, and Predicted Future Monthly Average Prices",
        labels={'value':'Average Price Per Night ($)', 'ds':'Date'})